In [1]:
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

In [2]:
from julep import Julep
import os
import yaml 
import time
from dotenv import load_dotenv

load_dotenv()

client = Julep(
    api_key=os.getenv('JULEP_API_KEY'),
    environment=os.getenv('JULEP_ENVIRONMENT')
)

In [3]:
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Julep Email",
    about=(
        "You are an agent that handles emails for julep users."
        + " Julep is a platform for creating kick-ass AI agents."
    ),
    model="claude-3.5-sonnet",
)

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [14]:
import yaml

MAILGUN_PASSWORD = os.environ['MAILGUN_PASSWORD']

task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: Julep Email Assistant
description: A Julep agent that can send emails and search the documentation.

########################################################
####################### INPUT SCHEMA ###################
########################################################
input_schema:
  type: object
  properties:
    from:
      type: string
    to:
      type: string
    subject:
      type: string
    body:
      type: string

                          
########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: send_email
  type: integration
  integration:
    provider: email
    setup:
      host: smtp.mailgun.org
      password: $ f'{MAILGUN_PASSWORD}'
      port: 587
      user: postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org

- name: search_docs
  type: system
  system:
    resource: agent
    subresource: doc
    operation: search
  
########################################################
####################### MAIN WORKFLOW ##################
########################################################

main:
# Step 0: Prompt the user for the email details
- prompt: |-
    $ f'''You are {{ agent.name }}. {{ agent.about }}

    A user with email address {{ _.from }} has sent the following inquiry:
    ------
      Subject: {{ _.subject }}

      {{ _.body }}
    ------

    Can you generate a query to search the documentation based on this email?
    Just respond with the query as is and nothing else.'''

  unwrap: true

# Step 1: Search the documentation
- tool: search_docs
  arguments:
    agent_id: {agent.id}
    text: $ _
    
- prompt: >-
    $ f'''You are {{ agent.name }}. {{ agent.about }}

    A user with email address {{ steps[0].input.from }} has sent the following inquiry:
    ------
      Subject: {{ steps[0].input.subject }}

      {{ steps[0].input.body }}
    ------

    Here are some possibly relevant snippets from the julep documentation:
    {{ '\\n'.join([snippet.content for doc in _.docs for snippet in doc.snippets]) }}
    
    ========

    Based on the above info, craft an email body to respond with as a json object.
    The json object must have `subject` and `body` fields.'''
  response_format:
    type: json_object
    
  unwrap: true

# Step 3: Extract the email
- evaluate:
    subject: $ extract_json(_)['subject']
    body: $ extract_json(_)['body']

# Step 4: Send the email
- tool: send_email
  arguments:
    body: $ _.body
    from: postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org
    subject: $  _.subject
    to: $ steps[0].input['from']
""")

<span style="color:olive;">Notes:</span>
- The `unwrap: True` in the prompt step is used to unwrap the output of the prompt step (to unwrap the `choices[0].message.content` from the output of the model).
- The `$` sign is used to differentiate between a Python expression and a string.
- The `_` refers to the output of the previous step.
- The `steps[index].input` refers to the input of the step at `index`.
- The `steps[index].output` refers to the output of the step at `index`.

In [15]:
task = client.tasks.create_or_update(
    agent_id=AGENT_UUID,
    task_id=TASK_UUID,
    **task_def,
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [16]:
execution = client.executions.create(
    task_id=task.id,
    input={"from": "postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org", "to": "akshat_s1@mt.iitr.ac.in", "subject": "what's up", "body": "sup"},
)

In [17]:
client.executions.get(execution.id)

Execution(id='067fc14a-fb21-7882-8000-4c91726ff0c2', created_at=datetime.datetime(2025, 4, 13, 19, 46, 55, 699894, tzinfo=datetime.timezone.utc), input={'to': 'akshat_s1@mt.iitr.ac.in', 'body': 'sup', 'from': 'postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org', 'subject': "what's up"}, status='queued', task_id='cefda6a3-52a0-4b80-894f-3a75e361faa2', updated_at=datetime.datetime(2025, 4, 13, 19, 46, 55, 699894, tzinfo=datetime.timezone.utc), error=None, metadata={}, output={}, transition_count=0)

### Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [18]:
execution_transitions = client.executions.transitions.list(
    execution_id=execution.id).items

for transition in reversed(execution_transitions):
    print("Type: ", transition.type)
    print("output: ", transition.output)
    print("-" * 100)

Type:  init
output:  {'to': 'akshat_s1@mt.iitr.ac.in', 'body': 'sup', 'from': 'postmaster@sandbox8fb6fbc34368497b877d285ae92db89e.mailgun.org', 'subject': "what's up"}
----------------------------------------------------------------------------------------------------
Type:  error
output:  EvaluateError: f-string: expecting '=', or '!', or ':', or '}' (<unknown>, line 3)
----------------------------------------------------------------------------------------------------
